# Task 7 - RSA Keygen
Krypto Lab

Felix Kleinsteuber, 185 709

In [1]:
import random
import numpy as np
import math
random.seed(1081)

# 1. Randomisierter Primzahltest (Miller-Rabin)
Der Miller-Rabin-Test ist ein randomisierter Primzahltest und gibt für Primzahlen sicher True, für zusammengesetzte Zahlen meist False zurück. Mit $p < \frac{1}{4}$ gibt der Test jedoch auch für zusammengesetzte Zahlen True zurück. Durch $n$-malige Ausführung des Tests kann eine beliebige Zahl mit der Sicherheit $p = 1 - \left(\frac{1}{4}\right)^n$ korrekt als Primzahl identifiziert werden (für $n=5: p > 0.999$, für $n = 10: p > 0.999999$).

In [2]:
# kopiert aus Übung 6
def quad_and_mult(x, m, n):
    """Berechnet effizient x^m mod n."""
    y = 1
    while m != 0:
        if m & 0x1 != 0:
            # falls bit=1: Multipliziere y mit x
            y = (y * x) % n
        # für jedes Bit wird x quadriert
        x = (x * x) % n
        # schiebe zum nächsten Bit
        m >>= 1
    return y

In [4]:
def miller_rabin_test(n):
    """Ist n prim? Gibt mit p < 1/4 True zurück, falls n zusammengesetzt ist (False Positive)
    und sicher True, falls n eine Primzahl ist."""
    assert n > 2
    # Bestimme ungerades m mit n - 1 = 2^k * m
    m = n - 1
    k = 0
    # Solange m gerade
    while m & 0b1 == 0:
        m >>= 1
        k += 1
    
    # Wähle zufälliges 2 <= a < n
    a = random.randrange(2, n)

    # b = a^m mod n
    b = quad_and_mult(a, m, n)

    if b == 1:
        return True
    for i in range(k):
        if b == n - 1:
            return True
        else:
            b = (b * b) % n
    return False

In [6]:
# Test miller_rabin_test
print(list(range(3,14)))
print([miller_rabin_test(n) for n in range(3, 14)])
# Da der Test nur einmal aufgerufen wird, könnten auch fehlerhafte Ergebnisse vorkommen! (daher kein assert)

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[True, False, True, False, True, False, False, False, True, False, True]


# 2. Schlüsselgenerierung
1. Wir generieren ausreichend große und unterschiedliche Primzahlen p, q mit $n = p \cdot q, \phi(n) = (p - 1) (q - 1)$. Ausreichend große Kandidaten werden durch wiederholte Ausführung des Miller-Rabin-Tests als Primzahlen identifiert.
2. Wir generieren ein zu $\phi(n)$ teilerfremdes $e$. Dass $e$ zu $\phi(n)$ teilerfremd ist, verifizieren wir mit dem Erweiterten Euklidischen Algorithmus und erhalten dabei den Koeffizienten $d$, sodass $d \cdot e \equiv 1 \mod \phi(n)$.
3. Der Public Key setzt sich aus $(e, n)$ und der Private Key aus $(d, n)$ zusammen.

Liegen $p$ und $q$ zu nahe beieinander, ist eine Attacke mit dem Verfahren der Differenz der Quadrate möglich (siehe weiter unten).

In [7]:
# kopiert aus Übung 6
def extended_euclidian(a, b):
    """ Berechnet ggT(a,b) = sa + tb = r und gibt r, s, t zurück. """
    r = [a, b]
    s = [1, 0]
    t = [0, 1]
    while r[-1] != 0:
        q = r[-2] // r[-1]
        r.append(r[-2] - q * r[-1])
        s.append(s[-2] - q * s[-1])
        t.append(t[-2] - q * t[-1])
    return r[-2], s[-2], t[-2]

In [8]:
def is_prime(n):
    """Gibt True zurück, wenn mit sehr hoher Wahrscheinlichkeit (0.999999) prim ist. """
    # bis zu 10-fache Ausführung von Miller-Rabin
    for i in range(10):
        if miller_rabin_test(n) == False:
            return False
    # mit p > 1 - (1/4)**10 = 0.999999... ist n prim
    return True

def look_for_prime(z):
    # Suche nach Primzahlen > 30 * z
    for i in [1, 7, 11, 13, 17, 19, 23, 29]:
        n = 30 * z + i
        if is_prime(n):
            return n
    # Keine Primzahl gefunden, erhöhe z
    return look_for_prime(z + 1)

def gen_pq(min_z=2**128, max_z=2**256):
    p = look_for_prime(random.randrange(min_z, max_z))
    q = look_for_prime(random.randrange(min_z, max_z))
    if p * 1.2 > q and q * 1.2 > p:
        # p und q sind zu nah beieinander, nochmal versuchen
        return gen_pq(min_z, max_z)
    else:
        return p, q

def find_de(phi_n, min_e=2*10, max_e=2**60):
    """Gibt ein encrypt-decrypt key pair d, e zurück, sodass d*e mod phi(n) = 1."""
    while True:
        e = random.randrange(min_e, max_e)
        # Überprüfe, ob e teilerfremd zu phi(n) und bestimme im gleichen Schritt Inverses von e mod phi(n)
        ggT, d, _ = extended_euclidian(e, phi_n)
        if ggT == 1:
            return d if d > 0 else phi_n + d, e

In [9]:
p, q = gen_pq()
n = p * q
phi_n = (p - 1) * (q - 1)
d, e = find_de(phi_n)
assert (d * e) % phi_n == 1
print("p:", p)
print("q:", q)
print("n:", n)
print("phi(n):", phi_n)
print("d:", d)
print("e:", e)

p: 3445385425009147875138849784065402885461426866561847874577207947473828540627051
q: 2829963465970887404523392248931896222670022211810909047393362227331737551932713
n: 9750314878964467090295182973806618332900357156722439522420521795121757960016524505050800296028376969617037714299112639349746869340849630776255942705979619363
phi(n): 9750314878964467090295182973806618332900357156722439522420521795121757960016518229701909315993097307375004717000004507900668496583927660206081137139887059600
d: 8139034288716078749087108087843653771418348790049310024258826424843867788778394567929973122559526208100065160496763629351403441497736929058932736848964162199
e: 733587815216161799


# 3. RSA Encrypt/Decrypt Test (Alles zusammen)
Wir ver- und entschlüsseln eine Testsequenz mit den generierten Public und Private Keys.

In [10]:
# kopiert aus Task 6
def rsa_encrypt(e, n, x):
    assert np.all(x < n)
    return np.array([quad_and_mult(x1, e, n) for x1 in x])

def rsa_decrypt(d, n, y):
    # äquivalent zu rsa_encrypt, aber der Übersichtlichkeit halber mit anderen Variablennamen
    assert np.all(y < n)
    return np.array([quad_and_mult(y1, d, n) for y1 in y])

In [11]:
# Alles zusammen
def generate_keys(min_z=2**128, max_z=2**256, min_e=2**10, max_e=2**60):
    p, q = gen_pq(min_z=min_z, max_z=max_z)
    n = p * q
    phi_n = (p - 1) * (q - 1)
    d, e = find_de(phi_n, min_e=min_e, max_e=max_e)
    return (e, n), (d, n)

# Test
print("Generiere ein Public-Key-Private-Key-Pair:")
print(generate_keys())

Generiere ein Public-Key-Private-Key-Pair:
((783751911744049273, 4787877286028231672376265365586519149519590774954818191847411957747125162854128620227651539548956454117595922625963560325750854584451937871279513101806040577), (3515802888762411672047729384114342319196041838565588267019964305425467055289981459110405606934625331966494081150041689516999678903161601996727627807093704009, 4787877286028231672376265365586519149519590774954818191847411957747125162854128620227651539548956454117595922625963560325750854584451937871279513101806040577))


In [12]:
# Test mit Encrypt/Decrypt
pubkey, privkey = generate_keys()
inp = np.array([42, 21])
enc = rsa_encrypt(*pubkey, inp)
dec = rsa_decrypt(*privkey, enc)
print(dec)
assert np.array_equal(inp, dec)

[42 21]


Funktioniert!

# 4. Verfahren der Differenz der Quadrate
Mit dem Verfahren der Differenz der Quadrate kann das Faktorisierungsproblem $p \cdot q = n$ für $p \approx q$ in kurzer Zeit gelöst werden. Wir wählen daher absichtlich nahe beieinander liegende $p$ und $q$ und suchen anschließend $u, w$ mit 
$$N = u^2 - w^2 \Leftrightarrow N = (u - w)(u + w).$$

Wir beginnen mit $u = \lceil \sqrt{N} \rceil$ und inkrementieren $u$, bis es einen Integer-Kandidaten $w$ für die Gleichung gibt. Dann kennen wir $p = u + w, q = u - w$ ($p, q$ vertauschbar).

In [13]:
def gen_pq_unsafe(min_z=2**128, max_z=2**256, z_diff = 10):
    # Generiere ein p wie üblich
    p = look_for_prime(random.randrange(min_z, max_z))
    # Generiere q nahe zu p (Unterscheidung um ca. 30 * z_diff)
    q = look_for_prime(p // 30 + z_diff)
    return p, q

eps = np.finfo(np.float32).eps

# Gibt den ganzzahligen Anteil der Wurzel zurück (Newton-Methode).
# np.sqrt und math.sqrt arbeiten beide mit Gleitkommazahlen und sind daher für große
# Zahlen sehr ungenau. Ab Python 3.8 gibt es math.isqrt(n).
def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

def is_square(k):
    return isqrt(k) ** 2 == k

# Test
p, q = gen_pq_unsafe()
print("diff between p, q:", q - p)
assert is_square(41) == False
assert is_square(36) == True

diff between p, q: 594


In [14]:
# Verfahren der Differenz der Quadrate: Löst das Faktorisierungsproblem, d.h. liefert p und q, sodass p * q = n.
def diff_squares(n):
    u = isqrt(n) + 1
    assert u * u >= n
    while not is_square(u * u - n):
        u += 1
    w = isqrt(u * u - n)
    return u + w, u - w


# Tests
p, q = gen_pq_unsafe()
print("p:", p)
print("q:", q)
print("diff between p, q:", q - p)
p_, q_ = diff_squares(p * q)
print("p':", p_)
print("q':", q_)
print("p'*q':", p_ * q_)
assert max(p, q) == p_ and min(p, q) == q_

p: 62217902190058217017864755123882278918766080813095468667600034260065544922137
q: 62217902190058217017864755123882278918766080813095468667600034260065544922497
diff between p, q: 360
p': 62217902190058217017864755123882278918766080813095468667600034260065544922497
q': 62217902190058217017864755123882278918766080813095468667600034260065544922137
p'*q': 3871067352931651077546609023113186990901417527937524830036949967244915139682108112796607107676956881002317531632050877333418851080800773297681541564616089


Funktioniert!